# PUC Rio | MVP | Machine Learning

Aluno: Joel Carneiro Dutra





## **Definição do Problema:**

Uma empresa que possui várias franquias de lojas, precisa prever a quantidade de produtos que cada franquia precisará para manter seus estoques otimizados. O objetivo é garantir que cada franquia tenha o estoque adequado para atender à demanda de seus clientes, minimizando ao mesmo tempo os custos de armazenamento e o risco de falta de produtos.

### **Passo 1: Coleta de Dados**

A empresa coleta dados históricos de vendas e estoque de cada uma das franquias, juntamente com informações sobre sazonalidade, promoções, dados geográficos, entre outros fatores que podem influenciar a demanda.

Esta etapa foi realizada por meio de uma extração no Google BigQuery, contendo os seguintes campos:

- **dt_venda** - Data em que a venda foi realizada
- **loja** - Código das franquias que realizaram as vendas
- **uf** - UF das franquias contendo Rio de Janeiro e São Paulo
- **produto** - Descrição do produto vendido (Produto X, Y e Z)
- **canal_venda** - Canal onde ocorreu a venda (Loja ou Site)
- **tipo_venda** - Tipo de venda sendo Promoção ou Regular
- **vlr_venda** - Valor total da venda
- **qt_venda** - Quantidade de itens vendidos

Atributos:
- **qt_dias_com_estoque** - Quantidade de dias em que a loja tinha estoque
- **qt_dias_sem_estoque** - Quantidade de dias em que a loja não tinha estoque
- **qt_dias_com_estoque_aberta** - Quantidade de dias em que a loja tinha estoque e estava aberta para venda
- **qt_dias_sem_estoque_aberta** - Quantidade de dias em que a loja não tinha estoque e estava aberta para venda
- **qt_dias_loja_fechada** - Quantidade de dias em que a loja esteve fechada
- **estoque_loja** - Quantidade de estoque da loja
- **habilitador** - Determina se a loja deverá ser abastecida caso tenha menos que 5 itens no estoque (variável 1 ou 0)

In [ ]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from matplotlib import cm
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

### **Passo 2: Carga dos Dados**

Nessa etapa fazemos a conexão do Colab ao dataset (Google Drive), fazendo a requisição do arquivo em formato .csv e a leitura do arquivo em um DataFrame.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

caminho_arquivo = '/content/drive/My Drive/PUC-Rio/base_historica.csv'

# Leitura do arquivo CSV em um DataFrame
dataset = pd.read_csv(caminho_arquivo)
dataset.head()

### **Passo 3: Análise dos Dados**

Nesta etapa é apresentado algumas informações do dataset, tais como:

- Volume de registros (shape);
- Tipo de dado de cada atributo (dtypes);
- Descrição dos campos (describe).

In [ ]:
# Mostra as dimensões do dataset
print(dataset.shape)

In [ ]:
# Mostra as informações do dataset
print(dataset.info())

In [ ]:
dataset.head()

In [ ]:
# Verifica o tipo de dataset de cada atributo
dataset.dtypes

In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
dataset.describe()

### **Passo 4: Pré-Processamento dos Dados**

Nesta etapa realizamos os seguintes processos:

- Verificação por campos nulos (isnull);
- Distribuição da classe (groupby);
- Tratamento de Missings e Limpeza (replace);
- Histograma e Matriz de Correlação (plot);
- Feature Selection (SelectKBest, Recursiva e ExtraTrees).

In [ ]:
# verificando nulls no dataset
dataset.isnull().sum()

In [ ]:
# distribuição da classe
print(dataset.groupby('habilitador').size())

In [ ]:
# salvando um novo dataset para tratamento de missings

# recuperando os nomes das colunas
col = list(dataset.columns)

# o novo dataset irá conter todas as colunas com exceção da última (classe)
atributos = dataset[col[0:-1]]

# substituindo os zeros por NaN
atributos.replace(0, np.nan, inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# removendo as colunas 'vlr_venda' e 'qt_venda'
atributos.drop(['vlr_venda', 'qt_venda'], axis=1, inplace= True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# substituindo os NaN por 0
atributos['qt_dias_sem_estoque'].fillna(0, inplace=True)
atributos['qt_dias_sem_estoque_aberta'].fillna(0, inplace=True)
atributos['estoque_loja'].fillna(0, inplace=True)

# substituindo os NaN de 'qt_dias_com_estoque' e 'qt_dias_com_estoque_aberta' pela mediana da coluna
atributos['qt_dias_com_estoque'].fillna(atributos['qt_dias_com_estoque'].median(), inplace=True)
atributos['qt_dias_com_estoque_aberta'].fillna(atributos['qt_dias_com_estoque_aberta'].median(), inplace=True)
atributos['qt_dias_loja_fechada'].fillna(atributos['qt_dias_loja_fechada'].median(), inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# Guardando o novo dataset para testes futuros
datasetSemMissings = atributos

# incluindo a coluna 'class' no novo dataset
datasetSemMissings['habilitador'] = dataset['habilitador']

# exibindo as primeiras linhas
datasetSemMissings.head()

In [ ]:
# Referenciando os atribuitos a uma nova variável
dataset_atributos = datasetSemMissings.loc[:, ['qt_dias_com_estoque', 'qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta', 'qt_dias_sem_estoque_aberta', 'qt_dias_loja_fechada', 'estoque_loja', 'habilitador']]
dataset_atributos.head()

In [ ]:
# Histograma
dataset_atributos.hist(figsize = (15,10))
plt.show()

In [ ]:
# Density Plot
dataset_atributos.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
sns.heatmap(dataset_atributos.corr(), annot=True, cmap='RdBu');

** Feature Selection **

In [ ]:
# Preparação dos dados para Feature Selection

# Separação em bases de treino e teste (holdout)
array = dataset_atributos.values
X = array[:,0:6] # atributos
y = array[:,6]   # classe (target)

In [ ]:
# SelectKBest

# Seleção de atributos com SelectKBest
best_var = SelectKBest(score_func=f_classif, k=3)

# Executa a função de pontuação em (X, y) e obtém os atributos selecionados
fit = best_var.fit(X, y)

# Reduz X para os atributos selecionados
features = fit.transform(X)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset_atributos.columns[0:7])

# Exibe as pontuações de cada atributos e os 4 escolhidos (com as pontuações mais altas)
np.set_printoptions(precision=3) # 3 casas decimais
print("\nScores dos Atributos Originais:", fit.scores_)
print("\nAtributos Selecionados:", best_var.get_feature_names_out(input_features=dataset_atributos.columns[0:6]))

In [ ]:
# Eliminação Recursiva de Atributos

# Criação do modelo
modelo = LogisticRegression(max_iter=200)

# Eliminação Recursiva de Atributos
rfe = RFE(modelo, n_features_to_select=3)
fit = rfe.fit(X, y)

# Print dos resultados
print("Atributos Originais:", dataset_atributos.columns[0:7])

# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados"
# e com valor 1 em "Ranking dos Atributos")
print("\nAtributos Selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)
print("\nNomes dos Atributos Selecionados: %s" % fit.get_feature_names_out(input_features=dataset_atributos.columns[0:6]))

In [ ]:
# Importância de Atributos com ExtraTrees

# Criação do modelo para seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X,y)

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset_atributos.columns[0:7])

# Exibe a pontuação de importância para cada atributo (quanto maior a pontuação, mais importante é o atributo).
print(modelo.feature_importances_)

Atributo com maior relevância durante as análises: **estoque_loja**

- SelectKBest: **698.377**
- ExtraTrees: **0.745**
- Recursiva: **True**

### **Passo 5: Separação em conjunto de Treino e Teste (Holdout)**

Nesta etapa foram realizados os seguintes processos:

- Validação cruzada
- Teste e comparação dos modelos

*Holdout com estratificação*

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = dataset_atributos.values
X = array[:,0:6]
y = array[:,6]

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed) # holdout sem estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # Validação cruzada

*Holdout sem estratificação*

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = dataset_atributos.values
X = array[:,0:6]
y = array[:,6]

X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # Validação cruzada

In [ ]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os módulos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('CART', DecisionTreeClassifier()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# Criando os modelos para o VotingClassifier
bases = []
model1 = DecisionTreeClassifier()
bases.append(('cart', model1))
model2 = SVC()
bases.append(('svm', model2))

In [ ]:
# Criando os ensembles e adicionando-os na lista de modelos
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))

In [ ]:
# Lista para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

### **Passo 6: Pipeline**

Nesta etapa foi feita a comparação entre os modelos:

- Original
- Padronizado
- Normalizado

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Lista para armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []

# Criando os elementos do pipeline

#Algoritmos que serão utilizados
cart = ('CART', DecisionTreeClassifier())
svm = ('SVM', SVC())
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

In [ ]:
#Montando os pipelines

# Dataset original
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('SVM-orig', Pipeline([svm])))
pipelines.append(('RF-orig', Pipeline([random_forest])))
pipelines.append(('ET-orig', Pipeline([extra_trees])))
pipelines.append(('GB-orig', Pipeline([gradient_boosting])))

# Dataset padronizado
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))
pipelines.append(('ET-padr', Pipeline([standard_scaler, extra_trees])))
pipelines.append(('GB-padr', Pipeline([standard_scaler, gradient_boosting])))

# Dataset normalizado
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))
pipelines.append(('ET-norm', Pipeline([min_max_scaler, extra_trees])))
pipelines.append(('GB-norm', Pipeline([min_max_scaler, gradient_boosting])))

# Executando os pipelines
for name, model in pipelines:
  cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std())
  print(msg)

In [ ]:
# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(20,6))
fig.suptitle('Comparação dos Modelos - Dataset original, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### **Passo 7: Avaliação do modelo**

Nesta etapa foi realizado a avaliação do modelo em conjunto de teste

In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = SVC(max_iter=200)
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

### **Passo 8: Aplicação em produção**

Nesta etapa, após a escolha do modelo, foi realizado o uso do mesmo com novos dados

In [ ]:
# Preparação do modelo com todo o dataset
scaler = StandardScaler().fit(X) # ajuste do scaler com todo o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com todo o dataset
model.fit(rescaledX, y)

In [ ]:
# Simulação com novos dados

data = {'qt_dias_com_estoque': [5, 30, 10],
        'qt_dias_sem_estoque': [3, 5, 15],
        'qt_dias_com_estoque_aberta': [2, 20, 10],
        'qt_dias_sem_estoque_aberta': [0, 5, 20],
        'qt_dias_loja_fechada': [1, 7, 15],
        'estoque_loja': [0, 2, 5],
        }

atributos = ['qt_dias_com_estoque', 'qt_dias_sem_estoque', 'qt_dias_com_estoque_aberta', 'qt_dias_sem_estoque_aberta', 'qt_dias_loja_fechada', 'estoque_loja']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:6].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

In [ ]:
# Predição de classes dos dados de entrada

saidas = model.predict(rescaledEntradaX)
print(saidas)

### **Conclusão**

...